In [40]:
import gzip
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import numpy as np
import nltk
#nltk.download ()
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

In [41]:
import string
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.stem import WordNetLemmatizer

In [42]:
df = pd.read_json("renttherunway_final_data.json.gz",lines = True)
df = shuffle(df)

In [43]:
total_df = df[["review_text","review_summary","rating","size","fit"]]
total_df["review_total"] = total_df["review_summary"]+ " " +total_df["review_text"]
total_df=total_df.drop(["review_text","review_summary"],axis=1)

/var/folders/kl/8j13qjxs4fd9fz6_ng5b4qpc0000gn/T/ipykernel_33359/201975335.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_df["review_total"] = total_df["review_summary"]+ " " +total_df["review_text"]


In [44]:
total_df = total_df.dropna(how='any',axis=0) 

In [45]:
def convert_fit(fit):
    if fit == 'large':
        out = 0
    elif fit == 'small':
        out = 1
    else:
        out = 2
    return out
total_df["fit"] = total_df["fit"].apply(lambda x: convert_fit(x))

In [46]:
def convert_rating(rating):
    return rating/2-1
total_df["rating"] = total_df["rating"].apply(lambda x: convert_rating(x))

In [47]:
stopwords = nltk.corpus.stopwords.words('english')
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
total_df['review_total']= total_df['review_total'].apply(lambda x:remove_punctuation(x))
def tokenization(text):
    tokens = word_tokenize(text)
    return tokens
#applying function to the column
total_df['review_total']= total_df['review_total'].apply(lambda x: tokenization(x))

stopwords = nltk.corpus.stopwords.words('english')

def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output
total_df['review_total']= total_df['review_total'].apply(lambda x:remove_stopwords(x))

wordnet_lemmatizer = WordNetLemmatizer()
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text
total_df['review_total']=total_df['review_total'].apply(lambda x:lemmatizer(x))

In [48]:
def combine(text):
    string = ''
    for i in range(len(text)):
        string += text[i]
        string += ' '
    return string
total_df['review_total']=total_df['review_total'].apply(lambda x:combine(x))

In [ ]:
total_df["sentiment_score"] = total_df["rating"]

In [ ]:
def convert_review(review):
    sent_dict = sid.polarity_scores(review)
    return sent_dict['compound']
total_df["sentiment_score"] = total_df["review_total"].apply(lambda x: convert_review(x))

In [ ]:
total_df=total_df.drop(["review_total"],axis=1)

In [ ]:
label_df = total_df[["rating"]]

In [ ]:
total_df=total_df.drop(["rating"],axis=1)

In [ ]:
X_train, X_valid,y_train, y_valid = train_test_split(total_df,label_df,test_size=0.2)

In [ ]:
xgb = XGBClassifier(max_depth=3,eta=0.3,silent=1,objective='multi:softprob',num_class=3)

In [ ]:
model = xgb.fit(X_train, y_train)

In [ ]:
fits = xgb.predict(X_train)
pred = xgb.predict(X_valid)
y_train_arr = y_train["rating"].to_numpy()
y_valid_arr = y_valid["rating"].to_numpy()
acc_xgbfits = (fits == y_train_arr).sum().astype(float) / len(fits)*100
acc_xgb = (pred == y_valid_arr).sum().astype(float) / len(pred)*100

In [ ]:
acc_xgb

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_classifier = KNeighborsClassifier()
knn_classifier.fit(X_train, y_train)
knn_predictions = knn_classifier.predict(X_valid)

In [ ]:
acc_knn = (knn_predictions == y_valid_arr).sum().astype(float) / len(knn_predictions)*100
acc_knn

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, y_train)
rf_predictions = rf_classifier.predict(X_valid)

In [ ]:
acc_rf = (rf_predictions == y_valid_arr).sum().astype(float) / len(rf_predictions)*100
acc_rf

In [ ]:
from sklearn.neural_network import MLPClassifier
snn_classifier = MLPClassifier()
snn_classifier.fit(X_train, y_train)
snn_predictions = snn_classifier.predict(X_valid)

In [ ]:
acc_snn = (snn_predictions == y_valid_arr).sum().astype(float) / len(snn_predictions)*100
acc_snn

In [ ]:
dnn_classifier = MLPClassifier(hidden_layer_sizes = [100]*5)
dnn_classifier.fit(X_train, y_train)
dnn_predictions = dnn_classifier.predict(X_valid)

In [ ]:
acc_dnn = (dnn_predictions == y_valid_arr).sum().astype(float) / len(dnn_predictions)*100
acc_dnn

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
dnns_classifier = OneVsRestClassifier(MLPClassifier(hidden_layer_sizes = [100]*5))
dnns_classifier.fit(np.array(X_train), y_train)
dnns_predictions_labels = dnns_classifier.predict(X_valid)

In [ ]:
acc_dnn = (dnns_predictions_labels == y_valid_arr).sum().astype(float) / len(dnns_predictions_labels)*100
acc_dnn

In [ ]:
from sklearn.svm import SVC
svm_classifier = SVC(decision_function_shape='ovr')
svm_classifier.fit(X_train, y_train)
svm_predictions_labels = svm_classifier.predict(X_valid)

In [ ]:
acc_svm = (svm_predictions_labels == y_valid_arr).sum().astype(float) / len(svm_predictions_labels)*100
acc_svm

In [ ]:
xgb_classifier = OneVsRestClassifier(XGBClassifier())
xgb_classifier.fit(X_train, y_train)
xbg_predictions_labels = xgb_classifier.predict(X_valid)

In [ ]:
acc_xgb = (xbg_predictions_labels == y_valid_arr).sum().astype(float) / len(xbg_predictions_labels)*100
acc_xgb

In [ ]:
total_df.head()

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [49]:
tf_idf = TfidfVectorizer()

label_df = total_df[["rating"]]
x_df = total_df[["review_total"]]

x_arr = x_df["review_total"].to_numpy()
y_arr = label_df["rating"].to_numpy()

X_train, X_valid,y_train, y_valid = train_test_split(x_arr,y_arr,test_size=0.2)

X_train_tf = tf_idf.fit_transform(X_train)

X_train_tf = tf_idf.transform(X_train)

X_test_tf = tf_idf.transform(X_valid)

In [50]:
xgb = XGBClassifier(max_depth=3,eta=0.3,silent=1,objective='multi:softprob',num_class=3)

model = xgb.fit(X_train_tf, y_train)

y_pred = xgb.predict(X_test_tf)

from sklearn import metrics

print(metrics.classification_report(y_valid, y_pred))

print(metrics.confusion_matrix(y_valid, y_pred))

[20:14:43] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1667849653518/work/src/learner.cc:767: 
Parameters: { "silent" } are not used.

              precision    recall  f1-score   support

         0.0       0.23      0.03      0.06       203
         1.0       0.30      0.05      0.08       575
         2.0       0.45      0.11      0.17      2184
         3.0       0.54      0.30      0.39     10608
         4.0       0.74      0.94      0.83     24923

    accuracy                           0.70     38493
   macro avg       0.45      0.29      0.31     38493
weighted avg       0.66      0.70      0.65     38493

[[    7    20    36    62    78]
 [    7    26   117   224   201]
 [    9    30   233  1005   907]
 [    7    10   109  3230  7252]
 [    1     0    19  1471 23432]]


In [57]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier

In [55]:
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf, y_train)
y_pred = naive_bayes_classifier.predict(X_test_tf)

print(metrics.classification_report(y_valid, y_pred))

print(metrics.confusion_matrix(y_valid, y_pred))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       203
         1.0       0.00      0.00      0.00       575
         2.0       0.00      0.00      0.00      2184
         3.0       0.46      0.04      0.07     10608
         4.0       0.66      1.00      0.79     24923

    accuracy                           0.66     38493
   macro avg       0.22      0.21      0.17     38493
weighted avg       0.56      0.66      0.53     38493

[[    0     0     0    32   171]
 [    0     0     0    90   485]
 [    0     0     0   272  1912]
 [    0     0     1   418 10189]
 [    0     0     0    90 24833]]


/Users/ranjangsk/opt/anaconda3/envs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ranjangsk/opt/anaconda3/envs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ranjangsk/opt/anaconda3/envs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

In [58]:
xgb_classifier = OneVsRestClassifier(XGBClassifier())
xgb_classifier.fit(X_train_tf, y_train)
y_pred = xgb_classifier.predict(X_test_tf)
print(metrics.classification_report(y_valid, y_pred))
print(metrics.confusion_matrix(y_valid, y_pred))

              precision    recall  f1-score   support

         0.0       0.27      0.05      0.09       203
         1.0       0.33      0.08      0.13       575
         2.0       0.43      0.13      0.20      2184
         3.0       0.54      0.35      0.43     10608
         4.0       0.75      0.93      0.83     24923

    accuracy                           0.71     38493
   macro avg       0.46      0.31      0.33     38493
weighted avg       0.67      0.71      0.67     38493

[[   11    26    48    68    50]
 [    8    45   150   240   132]
 [   15    51   282  1134   702]
 [    6    15   158  3728  6701]
 [    1     1    22  1763 23136]]
